In [19]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [20]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [21]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [22]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [25]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
numeric_columns = df_market_data.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()


df_scaled= df_market_data.copy()
df_scaled[numeric_columns] = scaler.fit_transform(df_market_data[numeric_columns])

df_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [28]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [29]:
# Create an empty list to store the inertia values
inertia = []
k_values = list(range(1, 11)) 

# Create a for loop to compute the inertia with each possible value of k
for i in k_values:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=i, random_state=0)
# 2. Fit the model to the data using `df_market_data_scaled`
    kmeans.fit(df_scaled)

# 3. Append the model.inertia_ to the inertia list
    inertia.append(kmeans.inertia_)

C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have

In [30]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_data = {
    'k': k ,
    'inertia': inertia
}

# Create a DataFrame from the dictionary

df_elbow = pd.DataFrame(elbow_curve_data)

# Create a DataFrame with the data to plot the Elbow curve
df_elbow.head()

,k,inertia
0,1,287.000000
1,2,198.571818
2,3,145.635748
3,4,79.022435
4,5,66.217780


In [31]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
# reference: finding_k_solution.ipynb
elbow_plot_original = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve - Original Scaled Data",
    xticks=k
)
elbow_plot_original

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [35]:
# Initialize the K-Means model using the best value for k
modelo = KMeans(n_clusters=4, random_state=0)

In [36]:
# Fit the K-Means model using the scaled DataFrame
modelo.fit(df_scaled)

C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [37]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predicted_clusters = modelo.predict(df_scaled)


# Print the resulting array of cluster values.
print(predicted_clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [38]:
# Create a copy of the scaled DataFrame
df_scaled_copy = df_scaled.copy()

In [39]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_scaled['predicted_cluster'] = predicted_clusters


# Display the copy of the scaled DataFrame
df_scaled_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [40]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.


---

### Optimize Clusters with Principal Component Analysis.

In [43]:
# Create a scatter plot using hvPlot by setting
df_scaled_copy['coin_id'] = df_market_data.index  # Add cryptocurrency names (coin_id)

# Step 2: Create the scatter plot
cluster_scatter_plot = df_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",   # x-axis: 24-hour price change percentage
    y="price_change_percentage_7d",    # y-axis: 7-day price change percentage
    c="predicted_cluster",             # Color points based on predicted clusters
    cmap='viridis',                    # Color map for better visualization
    hover_cols=['coin_id'],            # Show cryptocurrency name on hover
    title="Cryptocurrency Clusters Based on Price Changes",
    height=600, width=800
)
cluster_scatter_plot

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [44]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_result = pca.fit_transform(df_scaled)
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2', 'PC3'], index=df_scaled.index)



# View the scaled PCA data
df_pca.head()


,PC1,PC2,PC3
coin_id,,,
bitcoin,1.194082,-0.902074,-0.585338
ethereum,1.009706,-0.663584,-1.136950
tether,-0.722536,-0.307632,0.721813
ripple,-0.748266,-0.332379,0.558479
bitcoin-cash,2.258539,-1.826966,-1.378166


In [45]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_


print("Explained Variance Ratio by each principal component:", explained_variance)

Explained Variance Ratio by each principal component: [0.37269822 0.32489961 0.18917649]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [47]:
# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2', 'PC3'])



# Copy the crypto names from the original scaled DataFrame
df_pca['coin_id'] = df_market_data.index 


# Set the coin_id column as index
df_pca.set_index('coin_id', inplace=True)



# Display the scaled PCA DataFrame
df_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,1.194082,-0.902074,-0.585338
ethereum,1.009706,-0.663584,-1.136950
tether,-0.722536,-0.307632,0.721813
ripple,-0.748266,-0.332379,0.558479
bitcoin-cash,2.258539,-1.826966,-1.378166


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [50]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [51]:
# Create an empty list to store the inertia values
inertia_values = []

# Step 2: Loop through each k-value (from 1 to 11)
for k in range(1, 12):
    # Step 3: Create a KMeans model with the current k-value
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # Step 4: Fit the model to the PCA-reduced data
    kmeans.fit(df_pca)  # Use df_pca (PCA-reduced data)
    
    # Step 5: Append the inertia value to the list
    inertia_values.append(kmeans.inertia_)

# Print the inertia values for each k
print(inertia_values)


C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have

[290.0184568161517, 199.10805307785157, 112.4012011624448, 103.88808569697609, 35.060103024554635, 30.452705700916212, 19.649877928230485, 16.316381252463376, 14.786068344455987, 12.958588172813048, 8.903660951758685]


C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [52]:
# Create a dictionary with the data to plot the Elbow curve

# Create a DataFrame with the data to plot the Elbow curve
elbow_data = {
    'k': list(range(1, 12)),  
    'inertia': inertia_values  
}

# Step 2: Create a DataFrame from the dictionary
df_elbow = pd.DataFrame(elbow_data)

# Display the DataFrame
df_elbow.head()

,k,inertia
0,1,290.018457
1,2,199.108053
2,3,112.401201
3,4,103.888086
4,5,35.060103


In [53]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve - PCA Scaled Data", 
    xticks=k
)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k = 4 would be the best value for using PCA data because the inertia reduction slows at this point on.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [56]:

# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4, random_state=0)

In [57]:
# Fit the K-Means model using the PCA data
kmeans.fit(df_pca)

C:\Users\pangz\.conda\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [58]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
cluster_predictions = kmeans.predict(df_pca)

# Print the resulting array of cluster values.
print("Predicted clusters for each cryptocurrency:")
print(cluster_predictions)

Predicted clusters for each cryptocurrency:
[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 2 1 0 0 3
 0 0 0 0]


In [59]:
# Create a copy of the scaled PCA DataFrame
df_pca_copy = df_pca.copy()


# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_pca_copy['cluster'] = cluster_predictions


# Display the copy of the scaled PCA DataFrame
print(df_pca_copy.head())

                   PC1       PC2       PC3  cluster
coin_id                                            
bitcoin       1.194082 -0.902074 -0.585338        1
ethereum      1.009706 -0.663584 -1.136950        1
tether       -0.722536 -0.307632  0.721813        0
ripple       -0.748266 -0.332379  0.558479        0
bitcoin-cash  2.258539 -1.826966 -1.378166        1


In [60]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
cluster_plot_pca= df_pca_copy.hvplot.scatter(
    x='PC1', 
    y='PC2', 
    c='cluster',          # Color the points based on their cluster labels
    cmap='viridis',       # Choose a color map (you can change this to any other color map you like)
    size=10,              # Size of the points in the plot
    hover_cols=['coin_id'],  # Show the cryptocurrency name in the hover tooltip
    title="Cryptocurrency Clusters in PCA Space",  # Add a title to the plot
    xlabel="Principal Component 1 (PC1)",  # Label for x-axis
    ylabel="Principal Component 2 (PC2)",  # Label for y-axis
    width=800,            # Set the width of the plot
    height=600           # Set the height of the plot
)


cluster_plot_pca

:Scatter   [PC1]   (PC2,cluster,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [62]:
# Composite plot to contrast the Elbow curves
composite_elbow_plot = elbow_plot_original + elbow_plot_pca

composite_elbow_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [63]:
# Composite plot to contrast the clusters
composite_plot = cluster_scatter_plot + cluster_plot_pca

composite_plot

:Layout
   .Scatter.I  :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .Scatter.II :Scatter   [PC1]   (PC2,cluster,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of using fewer features to cluster the data using K-means is that the clusters become more distinct with less overlap in between the points.